In [ ]:
from ddnn.nn import *
from ddnn.validation import *
from ddnn.data import *

estimator = Estimator(
    net=NeuralNetwork(
        [
            LinearLayer((17, 4)),
            ActivationFunction("tanh"),
            LinearLayer((4, 1)),
            ActivationFunction("logistic"),
        ]
    ),
    loss=LossFunction("MSE"),
    # optimizer=Optimizer("SGD", learning_rate=1, momentum_coefficient=0.9, l2_coefficient=0),
    optimizer=Optimizer("Adam", learning_rate=1e-3, l2_coefficient=1e-2),
    batchsize=-1,
    initializer=Initializer("glorot_uniform"),
)
early_stopping = None
epochs = 500
dataset_type = ("monk", 3)
# dataset_type = "ML_cup"
log_every = 1
losses = ["MSE", "binary_accuracy"]

In [ ]:
if isinstance(dataset_type, tuple):
    traindata = read_monks(dataset_type[1], "train")
    traindata = onehot_encoding(data=traindata)

    testdata = read_monks(dataset_type[1], "test")
    testdata = onehot_encoding(data=testdata)
else:
    traindata, testdata = train_valid_split(read_ML_cup("train"), seed=123)

In [ ]:
print(traindata.shape, testdata.shape)

In [ ]:
testlogger = Logger(
    estimator,
    losses=losses,
    training_set=traindata,
    validation_set=testdata,
    every=log_every,
)
if early_stopping is not None:
    print("early stopping")
    teststopper = TrainingThresholdStopping(estimator, early_stopping)

    def callback(record):
        testlogger(record)
        teststopper(record)

else:

    def callback(record):
        testlogger(record)

In [ ]:
estimator.train(traindata, callback=callback, n_epochs=epochs)

In [ ]:
res = estimator.evaluate(losses=losses, dataset=traindata)
res

In [ ]:
res = estimator.evaluate(losses=losses, dataset=testdata)
res

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from ipywidgets import interact

In [ ]:
%matplotlib ipympl

# 1 plot with train and valid

fig, ax = plt.subplots()


@interact(
    loss=testlogger._losses,
)
def plot_results(loss):
    fig.tight_layout()
    ax.clear()
    for where, style in zip(["train", "valid"], [None, "dotted"]):
        y = testlogger._scores[0]["folds"][0][where][loss]
        if loss == "binary_accuracy":
            # todo fix to show last not best
            best = max(y)
            form = "{:.2}"
            logplot = False
        else:
            best = min(y)
            form = "{:.2E}"
            logplot = True
        # scale to resemble number of epochs instead of plot points
        ticks_x = ticker.FuncFormatter(
            lambda x, pos: "{0:g}".format(x * testlogger._every)
        )
        ax.xaxis.set_major_formatter(ticks_x)
        if logplot:
            ax.set_yscale("log")
        else:
            ax.set_yscale("linear")
        if where == "valid":
            where = "test"
        ax.plot(
            y, label=f"{where}: {form.format(best)}", linestyle=style, color="black"
        )
        ax.legend()

In [ ]:
n_tries = 30
train_loss_avg = {"MSE": 0, "binary_accuracy": 0}
test_loss_avg = {"MSE": 0, "binary_accuracy": 0}
for i in range(n_tries):
    estimator.reset()
    estimator.train(traindata, callback=lambda x: None, n_epochs=epochs)
    train_res = estimator.evaluate(losses=losses, dataset=traindata)
    test_res = estimator.evaluate(losses=losses, dataset=testdata)
    print(train_res, test_res)
    for loss in losses:
        train_loss_avg[loss] += train_res[loss]
        test_loss_avg[loss] += test_res[loss]

for loss in losses:
    train_loss_avg[loss] = train_loss_avg[loss] / n_tries
    test_loss_avg[loss] = test_loss_avg[loss] / n_tries

In [ ]:
print(train_loss_avg)
print(test_loss_avg)